In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv("../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv")

In [ ]:
data

In [ ]:
from sklearn.model_selection import train_test_split
data_latih,data_uji = train_test_split(data,test_size=0.2,random_state=101)

In [ ]:
print(data_uji.shape[0])
print(data_latih.shape[0])

In [ ]:
label_latih = data_latih.pop('DEATH_EVENT')
label_uji = data_uji.pop('DEATH_EVENT')

In [ ]:
data_uji1= data_uji.loc[:, ['age','ejection_fraction']]

In [ ]:
data_uji2= data_uji1.iloc[0:10]

In [ ]:
data_uji2

In [ ]:
X = data_uji2[["age", "ejection_fraction"]].to_numpy() 

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt

linked = linkage(X, 'average')

labelList = range(1, 11)

plt.figure(figsize=(10, 7))
dendrogram(linked,
            orientation='top',
            labels=labelList,
            distance_sort='descending',
            show_leaf_counts=True)
plt.show()

## 1. Proses Training

In [ ]:
def hitung_cost_gradient(W,X,Y,regularization) :
  jarak = 1 - (Y* np.dot(X,W))
  dw = np.zeros(len(W))
  if max(0,jarak)==0:
    di=W
  else:
    di = W - (regularization * Y*X)
    dw += di
  return dw

In [ ]:
from sklearn.utils import shuffle
def sgd(data_latih,label_latih,learning_rate = 0.000001,max_epoch=1000,regularization=10000):
  data_latih = data_latih.to_numpy()
  label_latih = label_latih.to_numpy()
  bobot = np.zeros(data_latih.shape[1])
  for epoch in range(1,max_epoch):
    X,Y = shuffle(data_latih,label_latih,random_state=101)
    for index,x in enumerate(X):
      delta = hitung_cost_gradient(bobot,x,Y[index],regularization)
      bobot = bobot - (learning_rate * delta)
    return bobot

In [ ]:
W = sgd(data_latih,label_latih)
print(W)

## 2. Proses Testing

In [ ]:
def testing(W,data_uji):
  prediksi = np.array([])
  for i in range(data_uji.shape[0]):
    y_prediksi = np.sign(np.dot(W,data_uji.to_numpy()[i]))
    prediksi = np.append(prediksi,y_prediksi)
  return prediksi

In [ ]:
y_prediksi = testing(W,data_uji)
print(sum(y_prediksi==label_uji))

# KNN

In [ ]:
def jarakEu(data1,data2):
  jarak = np.square(data1-data2)
  jarak = np.sum(jarak)
  return np.sqrt(jarak)

In [ ]:
jarak = jarakEu(data_latih.iloc[0],data_uji.iloc[0])
print(jarak)

In [ ]:
from collections import Counter
def knn(k,datalatih,labellatih,datauji):
  jarak = np.array([jarakEu(datalatih.iloc[x],datauji) for x in range(datalatih.shape[0])])
  indeks_k_minimum = jarak.argsort()[:k]
  k_kelas = labellatih.iloc[indeks_k_minimum].to_numpy()
  counter = Counter(k_kelas)
  kelas_uji = counter.most_common(1)[0][0]
  return kelas_uji

In [ ]:
hasil = knn(3,data_latih,label_latih,data_uji.iloc[1])
print(hasil)

In [ ]:
def knn_multi(k,datalatih,labellatih,datauji):
  kelasuji = np.array([knn(k,datalatih,labellatih,datauji.iloc[x])for x in range(datauji.shape[0])])
  return kelasuji

In [ ]:
def cek_hasil(prediksi, kebenaran):
  return sum(prediksi==kebenaran)

In [ ]:
prediksi1 = knn_multi(3, data_latih, label_latih, data_uji)

In [ ]:
cek_hasil(prediksi1, label_uji)

# CLUSTERING

In [ ]:
kelas = data.pop('DEATH_EVENT')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.pairplot(data,hue='DEATH_EVENT')

In [ ]:
def minmax(df_input):
  list_fitur = df_input.columns[:-1]
  for fitur in list_fitur:
    max = df_input[fitur].max()
    min = df_input[fitur].min()
    df_input[fitur] = (df_input[fitur]-min)/(max-min)
  return df_input